# Circuit Compilers Advanced Topics

Recall a circuit compiler is a function that takes an arithmetic circuit and compiles it into a program, specifically a `RawProgram` object.

There are several circuit compilers provided by CK, and it is possible to write custom circuit compilers. A circuit compiler is a callable with the signature:
```
def my_circuit_compiler(
    *result: CircuitNode,
    input_vars: InputVars = InferVars.ALL,
    circuit: Optional[Circuit] = None,
) -> RawProgram:
```

That is, the callable takes zero or more arguments, the circuit result nodes, and optional keyword arguments. The results nodes must be from the same circuit.

Parameter `input_vars` specifies how to determine the function input variables. Default is to use all circuit variables, in index order. Other options are documented in the module `ck.circuit_compiler.support.input_vars`.

Parameter `circuit` is rarely needed as each result node keeps track of the circuit it belongs to. However, in some circumstances, when there are no result nodes, the circuit needs to be provided. If the `circuit` parameter is used, then the supplied circuit must be the same as that of the result nodes.


CK provides many circuit compilers, each using different algorithms. Each provided circuit compiler is a `NamedCircuitCompiler` enum member.

Here are the named circuit compilers, which are explained in the next sections.

In [1]:
from ck.circuit_compiler import NamedCircuitCompiler

for compiler in NamedCircuitCompiler:
    print(compiler.name)

LLVM_STACK
LLVM_TMPS
LLVM_VM
CYTHON_VM
INTERPRET


## LLVM_STACK
Use the LLVM compiler to compile to a native binary function, where no temporary working memory is explicitly allocated at compile time or requested at run time. All temporary variables are allocated on the stack as determined by the LLVM compiler.

This compiler creates an extremely efficient run time. However, the compile time can be prohibitive for even moderately sized circuits.

## LLVM_TMPS
Use the LLVM compiler to compile to a native binary function, where  temporary working memory is allocated at compile time.

##  LLVM_VM
Use the LLVM compiler to compile a virtual CPU as native binary function, where  instructions for the virtual CPU are determined by traversing the circuit and stored as a constant array by the LLVM compiler.

This compiler creates a moderately efficient run time. The compile times can be significantly better than `LLVM_STACK` and `LLVM_TMPS`.

##  CYTHON_VM
Use a Cython implementation of a virtual CPU as native binary function, where  instructions for the virtual CPU are determined by traversing the circuit and provided to the Cythonised virtual CPU by the raw program.

This compiler creates a moderately efficient run time. The compile times are generally very fast, and are significantly better than LLVM compilers.

##  INTERPRET
Use a Python implementation of a virtual CPU as native binary function, where  instructions for the virtual CPU are determined by traversing the circuit and provided to the virtual CPU by the raw program.

This compiler creates an inefficient run time, but is easy to inspect and debug (As it is Python). The compile times are generally very fast.

Here is a demonstration of the named circuit compilers. This code show the compile time and program execution time for each compiler, using a circuit created from an example PGM.

In [2]:
import timeit
from ck.example import Insurance
from ck.pgm_compiler import DEFAULT_PGM_COMPILER
from ck.pgm_circuit import PGMCircuit
from ck.circuit import CircuitNode
from ck.program.program_buffer import ProgramBuffer

pgm = Insurance()
pgm_cct: PGMCircuit = DEFAULT_PGM_COMPILER(pgm)
top: CircuitNode = pgm_cct.circuit_top

for compiler in NamedCircuitCompiler:
    # Time compilation
    start_time = timeit.default_timer()
    raw_program = compiler(top)
    stop_time = timeit.default_timer()
    compile_time = (stop_time - start_time) * 1000  # as milliseconds

    # Time c running the program
    program = ProgramBuffer(raw_program)
    start_time = timeit.default_timer()
    program.compute()
    stop_time = timeit.default_timer()
    run_time = (stop_time - start_time) * 1000  # as milliseconds

    print(f'{compiler.name:>10}  {compile_time:8.3f}ms {run_time:8.3f}ms')


LLVM_STACK  2606.190ms    0.017ms


 LLVM_TMPS  3030.707ms    0.057ms
   LLVM_VM    90.379ms    0.174ms
 CYTHON_VM    30.663ms    0.145ms
 INTERPRET    37.358ms   18.122ms


The default circuit compiler is available as `DEFAULT_CIRCUIT_COMPILER`, which is a `NamedCircuitCompiler` enum member.

In [3]:
from ck.circuit_compiler import DEFAULT_CIRCUIT_COMPILER

DEFAULT_CIRCUIT_COMPILER.name

'CYTHON_VM'